In [2]:
import numpy as np
import pandas as pd

# Our functions
import EvalFunctions
import RecAlgs

The parameters to select how to tun the algorithm

In [1]:
# General
AmountOfRecommendations = 0     # The amount of 'good' recommendation that will be returned
TimeCutOffDays = 3              # How old the articles can be that we would consider recommending (older than X days are not considered)

# Data selection                
TrainingDataStartDate = 0       # From what day we want to collect data
TrainingDataWindowSize = 2      # How many days of training data we want, 0 for all
TestDataWindowSize = 1          # How many days of test data we want, 0 for all

# Algorithm specifics 
TypeOfRecAlg = 0                # Which RecAlg we want to use 0-Pop, 1-Rand, 2-CBF, 3-CF, 4-Hybrid

# Popular Baseline
TimePenaltyPerHour = 0.1        # The percentage on penalty per hour the news gets
TimePenaltyStart = 24           # After howmany hours in the past the penalty starts

# Random Baseline
MinScore = 0                    # Minimum score that can be given
MaxScore = 1                    # Maximum score that can be given

# Content based filtering

# Collaborative filtering

# Hybrid
UsePopBaseline = False          # Whether to use Popularity baseline
UseRandBaseLine = False         # Whether to use Random baseline
UseCBF = True                   # Whether to use Content based filtering
UseCF = True                    # Whether to use Collaborative filtering
TakeMax = False                 # Whether to take the max between CBF and CF before applying weights
Weights = [0.2, 0.4, 0.4]       # The weights for the different parts (in order of appearance above)


Data selection

In [4]:
import os
print(os.listdir("../data/"))  # Check available files in the folder

['MINDlarge_dev', 'MINDlarge_test', 'MINDlarge_train', 'MINDsmall_dev', 'MINDsmall_train']


In [14]:
# # Add the first time the article has been seen in the behaviors as the Est_PublishedTime in the articles.
# AllTrainingData = pd.read_csv("../data/MINDsmall_train/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
# AllValidationData = pd.read_csv("../data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
# AllData = pd.concat([AllTrainingData, AllValidationData], ignore_index=True)
# 
# ArticlesTrain = pd.read_csv("../data/MINDsmall_train/news.tsv", sep="\t", header=None, names=["ArticleID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
# ArticlesValidation = pd.read_csv("../data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["ArticleID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
# AllArticles = pd.concat([ArticlesTrain, ArticlesValidation], ignore_index=True)



In [18]:
# Add the first time the article has been seen in the behaviors as the Est_PublishedTime in the articles.
AllTrainingData = pd.read_csv("../data/MINDsmall_train/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
AllValidationData = pd.read_csv("../data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
AllData = pd.concat([AllTrainingData, AllValidationData], ignore_index=True)

ArticlesTrain = pd.read_csv("../data/MINDsmall_train/news.tsv", sep="\t", header=None, names=["ArticleID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
ArticlesValidation = pd.read_csv("../data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["ArticleID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
AllArticles = pd.concat([ArticlesTrain, ArticlesValidation], ignore_index=True)

# Convert timestamp to datetime if it's not already
AllData['DateTime'] = pd.to_datetime(AllData['DateTime'])

# Assume 'clicked_articles' is the column containing space-separated article IDs
AllData["ClickData"] = AllData["ClickData"].astype(str)
AllData = AllData.assign(ClickData=AllData["ClickData"].str.split(" ")).explode("ClickData")
# Explode ClickData: Each article ID gets its own row
AllData = AllData.explode("ClickData")

# Remove the last 2 characters from ClickData
AllData["ClickData"] = AllData["ClickData"].astype(str).str[:-2]
print(AllData["ClickData"].nunique())
print(AllArticles["ArticleID"].nunique())

# # Rename for clarity
# AllData.rename(columns={"ClickData": "ArticleID"}, inplace=True)
# 
# AllData["ArticleID"] = AllData["ArticleID"].str.strip().astype(str)
# AllArticles["ArticleID"] = AllArticles["ArticleID"].str.strip().astype(str)
# 
# # Find first appearance of each article
# article_publish_times = AllData.groupby("ArticleID")["DateTime"].min().reset_index()
# article_publish_times.rename(columns={"DateTime": "EstimatedPublishTime"}, inplace=True)
# 
# # Merge the estimated published times into AllArticles
# AllArticles = AllArticles.merge(article_publish_times, on="ArticleID", how="left")
# 
# AllArticles


22771
65238


,ArticleID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
5,N2073,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."
...,...,...,...,...,...,...,...,...
93682,N32558,news,elections-2020-us,Trump campaign launching black outreach effort...,"WASHINGTON (AP) During the 2016 campaign, ca...",https://assets.msn.com/labs/mind/BBWsd7A.html,"[{""Label"": ""Donald Trump"", ""Type"": ""P"", ""Wikid...","[{""Label"": ""Donald Trump"", ""Type"": ""P"", ""Wikid..."
93683,N12470,news,newscrime,3 teens shot in Northeast DC Thursday,"Thursday was a violent night in the District, ...",https://assets.msn.com/labs/mind/BBWse0x.html,"[{""Label"": ""Northeast (Washington, D.C.)"", ""Ty...","[{""Label"": ""Metropolitan Police Department of ..."
93684,N25642,news,newspolitics,Texas custody battle fuels debate over transge...,A custody battle between two parents has led t...,https://assets.msn.com/labs/mind/BBWseUG.html,"[{""Label"": ""Texas"", ""Type"": ""G"", ""WikidataId"":...","[{""Label"": ""Ted Cruz"", ""Type"": ""P"", ""WikidataI..."
93685,N20845,news,newsus,Police say 23-year-old man allegedly fled stat...,"HALL COUNTY, Ga. (CBS46) -- Fourteen-year-old ...",https://assets.msn.com/labs/mind/BBWsf0T.html,[],"[{""Label"": ""Hall County, Georgia"", ""Type"": ""G""..."


In [ ]:


if TrainingDataWindowSize == 0:
    TrainingData = AllTrainingData
else :
    TrainingData = 0
    

